In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ==============================
# Fungsi Keanggotaan Permintaan
perm_turun = lambda x: 1 if x <= 1000 else (3000 - x)/2000 if x <= 3000 else 0
perm_naik  = lambda x: 0 if x <= 1000 else (x - 1000)/2000 if x <= 3000 else 1

# Fungsi Keanggotaan Persediaan
pers_sedikit = lambda x: 1 if x <= 200 else (400 - x)/200 if x <= 400 else 0
pers_sedang  = lambda x: 0 if x <= 200 else (x - 200)/200 if x <= 400 else (800 - x)/400 if x <= 800 else 0
pers_banyak  = lambda x: 0 if x <= 500 else (x - 500)/300 if x <= 800 else 1

# Fungsi Keanggotaan Produksi
prod_berkurang = lambda x: 1 if x <= 2000 else (7000 - x)/5000 if x <= 7000 else 0
prod_bertambah = lambda x: 0 if x <= 2000 else (x - 2000)/5000 if x <= 7000 else 1

# ==============================
# Plot Fungsi Keanggotaan

x_perm = np.linspace(0, 5000, 100)
x_pers = np.linspace(0, 1000, 100)
x_prod = np.linspace(0, 8000, 100)

# Plot Permintaan
plt.figure(figsize=(6,4))
plt.plot(x_perm, [perm_turun(x) for x in x_perm], label='Turun')
plt.plot(x_perm, [perm_naik(x) for x in x_perm], label='Naik')
plt.title('Fungsi Permintaan')
plt.xlabel('Permintaan')
plt.ylabel('μ(x)')
plt.legend()
plt.grid(True)
plt.show()

# Plot Persediaan
plt.figure(figsize=(6,4))
plt.plot(x_pers, [pers_sedikit(x) for x in x_pers], label='Sedikit')
plt.plot(x_pers, [pers_sedang(x) for x in x_pers], label='Sedang')
plt.plot(x_pers, [pers_banyak(x) for x in x_pers], label='Banyak')
plt.title('Fungsi Persediaan')
plt.xlabel('Persediaan')
plt.ylabel('μ(x)')
plt.legend()
plt.grid(True)
plt.show()

# Plot Produksi
plt.figure(figsize=(6,4))
plt.plot(x_prod, [prod_berkurang(x) for x in x_prod], label='Berkurang')
plt.plot(x_prod, [prod_bertambah(x) for x in x_prod], label='Bertambah')
plt.title('Fungsi Produksi')
plt.xlabel('Produksi')
plt.ylabel('μ(x)')
plt.legend()
plt.grid(True)
plt.show()

# ==============================
# Input Data
a = 3000  # Permintaan
b = 700   # Persediaan

# ==============================
# Fuzzifikasi
mu_perm_turun   = perm_turun(a)
mu_perm_naik    = perm_naik(a)
mu_pers_sedikit = pers_sedikit(b)
mu_pers_sedang  = pers_sedang(b)
mu_pers_banyak  = pers_banyak(b)

print("=== Hasil Fuzzifikasi ===")
print(f"μ_perm_turun   = {mu_perm_turun:.3f}")
print(f"μ_perm_naik    = {mu_perm_naik:.3f}")
print(f"μ_pers_sedikit = {mu_pers_sedikit:.3f}")
print(f"μ_pers_sedang  = {mu_pers_sedang:.3f}")
print(f"μ_pers_banyak  = {mu_pers_banyak:.3f}")

# ==============================
# Aturan Fuzzy (6 Aturan)
# 1. IF Permintaan TURUN And Persediaan BANYAK THEN Produksi BERKURANG
# 2. IF Permintaan TURUN And Persediaan SEDANG THEN Produksi BERKURANG
# 3. IF Permintaan TURUN And Persediaan SEDIKIT THEN Produksi BERTAMBAH
# 4. IF Permintaan NAIK And Persediaan BANYAK THEN Produksi BERKURANG
# 5. IF Permintaan NAIK And Persediaan SEDANG THEN Produksi BERTAMBAH
# 6. IF Permintaan NAIK And Persediaan SEDIKIT THEN Produksi BERTAMBAH

r1 = min(mu_perm_turun, mu_pers_banyak)   # berkurang
r2 = min(mu_perm_turun, mu_pers_sedang)   # berkurang
r3 = min(mu_perm_turun, mu_pers_sedikit)  # bertambah
r4 = min(mu_perm_naik,  mu_pers_banyak)   # berkurang
r5 = min(mu_perm_naik,  mu_pers_sedang)   # bertambah
r6 = min(mu_perm_naik,  mu_pers_sedikit)  # bertambah

print("\n=== Derajat Kekuatan Aturan ===")
print(f"R1 (Turun & Banyak → Berkurang)   = {r1:.3f}")
print(f"R2 (Turun & Sedang → Berkurang)   = {r2:.3f}")
print(f"R3 (Turun & Sedikit → Bertambah)  = {r3:.3f}")
print(f"R4 (Naik & Banyak → Berkurang)    = {r4:.3f}")
print(f"R5 (Naik & Sedang → Bertambah)    = {r5:.3f}")
print(f"R6 (Naik & Sedikit → Bertambah)   = {r6:.3f}")

# ==============================
# Inferensi (Agregasi)
x = np.linspace(0, 8000, 16001)

# Gabungkan semua aturan berdasarkan konsekuen
clip_berkurang = np.minimum([prod_berkurang(xx) for xx in x], max(r1, r2, r4))
clip_bertambah = np.minimum([prod_bertambah(xx) for xx in x], max(r3, r5, r6))

# Agregasi
agg = np.maximum(clip_berkurang, clip_bertambah)

# ==============================
# Defuzzifikasi (Metode Centroid)
num = np.trapezoid(x * agg, x)
den = np.trapezoid(agg, x)
centroid = num / den

print(f"\n=== Hasil Defuzzifikasi ===")
print(f"Hasil defuzzifikasi = {centroid:.2f}")
print(f"Produksi yang disarankan = {centroid:.0f} kemasan")

# ==============================
# Visualisasi Agregasi Hasil
plt.figure(figsize=(8,4))
plt.plot(x, [prod_berkurang(xx) for xx in x], label='Berkurang (original)')
plt.plot(x, [prod_bertambah(xx) for xx in x], label='Bertambah (original)')
plt.fill_between(x, agg, color='gray', alpha=0.4, label='Agregasi Fuzzy')
plt.title('Hasil Agregasi dan Defuzzifikasi')
plt.xlabel('Produksi')
plt.ylabel('μ(x)')
plt.legend()
plt.grid(True)
plt.show()
